In [ ]:
Un problème de vp ???
#image_2019-06-15_10-47-26_1112_64_2.jpg

In [1]:
#D'abord vérifier si on a du Lenet

In [20]:
# import the necessary packages
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
import tensorflow as tf
import numpy as np
import time
import sys
import os
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import linear_model, preprocessing 
from keras.applications.imagenet_utils import preprocess_input
from sklearn import metrics
%matplotlib inline


In [21]:
def build_model1 (width, height, depth, classes):
	# initialize the input shape and channels dimension to be
	# "channels last" ordering
	inputShape = (height, width, depth)
	chanDim = -1
	# build the model using Keras' Sequential API
	model = Sequential([
		# CONV => RELU => BN => POOL layer set
		Conv2D(16, (3, 3), padding="same", input_shape=inputShape),
		Activation("relu"),
		BatchNormalization(axis=chanDim),
		MaxPooling2D(pool_size=(2, 2)),
		# (CONV => RELU => BN) * 2 => POOL layer set
		Conv2D(32, (3, 3), padding="same"),
		Activation("relu"),
		BatchNormalization(axis=chanDim),
		Conv2D(32, (3, 3), padding="same"),
		Activation("relu"),
		BatchNormalization(axis=chanDim),
		MaxPooling2D(pool_size=(2, 2)),
		# (CONV => RELU => BN) * 3 => POOL layer set
		Conv2D(64, (3, 3), padding="same"),
		Activation("relu"),
		BatchNormalization(axis=chanDim),
		Conv2D(64, (3, 3), padding="same"),
		Activation("relu"),
		BatchNormalization(axis=chanDim),
		Conv2D(64, (3, 3), padding="same"),
		Activation("relu"),
		BatchNormalization(axis=chanDim),
		MaxPooling2D(pool_size=(2, 2)),
		# first (and only) set of FC => RELU layers
		Flatten(),
		Dense(256),
		Activation("relu"),
		BatchNormalization(),
		Dropout(0.5),
		# softmax classifier
		Dense(classes),
		Activation("softmax")
	])
	# return the built model to the calling function
	return model

In [22]:
#Essayons de faire lenet avec tf voir ci-dessous . Il y a aussi une commande pour voir le résumer peut être .resume
def build_model2(width, height, depth, classes,drop_out_rate):
	# initialize the input shape and channels dimension to be
	# "channels last" ordering
	inputShape = (height, width, depth)
	chanDim = -1
	# build the model using Keras' Sequential API
	model = Sequential([
		# CONV => RELU => BN => POOL layer set
		Conv2D(30, (5, 5), padding="same", input_shape=inputShape),
		Activation("relu"),
		BatchNormalization(axis=chanDim),
		MaxPooling2D(pool_size=(2, 2)),
        
		# (CONV => RELU => BN) * 2 => POOL layer set
		Conv2D(16, (3, 3), padding="same"),
		Activation("relu"),
		BatchNormalization(axis=chanDim),
		MaxPooling2D(pool_size=(2, 2)),

        
		# first (and only) set of FC => RELU layers
		Flatten(),
		Dense(128),
		Activation("relu"),
		BatchNormalization(),
		Dropout(drop_out_rate),
        
		# softmax classifier
		Dense(classes),
		Activation("softmax")
	])
	# return the built model to the calling function
	return model

In [6]:
#Paramètres

#generateur_path='/mnt/VegaSlowDataDisk/c3po/Chaine_de_traitement/Train_imagettes_annotées/type_oiseau/Materiel/generateur.csv'
test_size

In [6]:
=0.2
epochs=200
batch_size = 600
zoom_range = 1.25
horizontal_flip = True
Minimum_Number_Class=100
dropout_rate=0.3
#steps_per_epoch=len(data_train)//batch_size
steps_per_epoch=1
#validation_steps=len(data_test)//batch_size
validation_steps=1

In [7]:

base_img_paths="/home/marcpozzo/Desktop/c3po/Images_aquises/"
generateur_path='/mnt/VegaSlowDataDisk/c3po/Images_aquises/generateur_bigger.csv'

df=pd.read_csv(generateur_path)
df.drop('labels',inplace=True,axis=1)

df["class"].unique()
for c in df:
    print(''+c+':',len(df[c].unique()))
df["class"].unique()


All_Unique=df["class"].unique()
Utilisable=[]
for i in df["class"].unique():
    if df["class"][df["class"]==i].count()>Minimum_Number_Class:
        Utilisable.append(i)
Utilisable
Non_Utilisable=set(All_Unique)-set(Utilisable)
Non_Utilisable
for i in Non_Utilisable:
    df=df[df["class"]!=i]
df=df[df["class"]!="oiseau"]  
df["class"].unique()


for i in range(len(df["class"])):
    image_name=df["img_paths"].iloc[i]
    df["img_paths"].iloc[i]=os.path.join(base_img_paths,image_name)
    

class: 13
img_paths: 6220


In [8]:
n_path="/mnt/VegaSlowDataDisk/c3po_interface/bin/fp_images/"



fp_new=pd.read_csv("/mnt/VegaSlowDataDisk/c3po_interface/bin/fp_threshold/table_fp_new.csv")

liste_img_paths=[]
for i in range(len(fp_new)):
    liste_img_paths.append(n_path+fp_new["imagetteName"].iloc[i])

fp_new["img_paths"]=liste_img_paths
#imagetteNamefp_new.head()

fp_new["class"]="autre"
fp_new.columns

to_drop=['path', 'filename', 'imagetteName', 'max_cat', 'cat', 'xmin', 'xmax','ymin', 'ymax', 'former_index']

fp_new.drop(to_drop,axis=1,inplace=True)

df=pd.concat([df,fp_new])

/home/marcpozzo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [29]:
from numpy.random import seed
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
seed(1)
tf.random.set_seed(2)

data_train,data_test= train_test_split(df,stratify=df["class"], test_size=test_size,random_state=42)
train_data_generator = ImageDataGenerator(
        preprocessing_function = preprocess_input
        # data augmentation
        #rotation_range = 10,
        #zoom_range = zoom_range,
        #horizontal_flip = horizontal_flip
        )

test_data_generator = ImageDataGenerator(preprocessing_function = preprocess_input)
    #preprocessing_function = preprocess_input)




train_generator = train_data_generator.flow_from_dataframe(dataframe=data_train,
                                                          directory="",
                                                           x_col = "img_paths",
                                                           class_mode ="sparse",
                                                          target_size = (28 , 28), 
                                                          batch_size = len(data_train) )


test_generator = test_data_generator.flow_from_dataframe(dataframe=data_test,
                                                          directory="",
                                                           x_col = "img_paths",
                                                           class_mode ="sparse",
                                                          target_size = (28 , 28), 
                                                          batch_size = len(data_test))

gen=train_generator[0]

x_train=gen[0]
y_train=gen[1]



gen_test=test_generator[0]

x_test=gen_test[0]
y_test=gen_test[1]

Found 40827 validated image filenames belonging to 6 classes.
Found 10207 validated image filenames belonging to 6 classes.


In [23]:
model = build_model1(28, 28, 3, 6,0.3)
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy",metrics=["acc"])
history=model.fit(x_train,y_train,epochs=200,validation_data=(x_test,y_test),verbose=0)
y_predict=model.predict(x_test).argmax(axis=1)
print(metrics.classification_report(y_predict,y_test,digits=5))

TypeError: build_model1() takes 4 positional arguments but 5 were given

In [25]:
#Test avec 2 epochs
model = build_model2(28, 28, 3, 6,0.3)
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy",metrics=["acc"])
history=model.fit(x_train,y_train,epochs=2,validation_data=(x_test,y_test),verbose=0)
y_predict=model.predict(x_test).argmax(axis=1)
print(metrics.classification_report(y_predict,y_test,digits=5))
history=model.fit(x_train,y_train,epochs=2,validation_data=(x_test,y_test),verbose=0)
y_predict=model.predict(x_test).argmax(axis=1)
print(metrics.classification_report(y_predict,y_test,digits=5))

/home/marcpozzo/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


              precision    recall  f1-score   support

         0.0    0.99392   0.97848   0.98614      9850
         1.0    0.00000   0.00000   0.00000         0
         2.0    0.63256   0.78161   0.69923       174
         3.0    0.40299   0.79412   0.53465        34
         4.0    0.02857   1.00000   0.05556         1
         5.0    0.66265   0.74324   0.70064       148

    accuracy                        0.97110     10207
   macro avg    0.45345   0.71624   0.49604     10207
weighted avg    0.98089   0.97110   0.97551     10207

              precision    recall  f1-score   support

           0    0.99742   0.98143   0.98936      9855
           1    0.51852   0.93333   0.66667        15
           2    0.67442   0.88415   0.76517       164
           3    0.46269   0.96875   0.62626        32
           4    0.31429   0.73333   0.44000        15
           5    0.64458   0.84921   0.73288       126

    accuracy                        0.97776     10207
   macro avg    0.60198

In [26]:
model = build_model2(28, 28, 3, 6,0.3)
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy",metrics=["acc"])

history=model.fit(x_train,y_train,epochs=100,validation_data=(x_test,y_test),verbose=0)
y_predict=model.predict(x_test).argmax(axis=1)
print(metrics.classification_report(y_predict,y_test,digits=5))
history=model.fit(x_train,y_train,epochs=100,validation_data=(x_test,y_test),verbose=0)
y_predict=model.predict(x_test).argmax(axis=1)
print(metrics.classification_report(y_predict,y_test,digits=5))



history=model.fit(x_train,y_train,epochs=100,validation_data=(x_test,y_test),verbose=0)
y_predict=model.predict(x_test).argmax(axis=1)
print(metrics.classification_report(y_predict,y_test,digits=5))
history=model.fit(x_train,y_train,epochs=100,validation_data=(x_test,y_test),verbose=0)
y_predict=model.predict(x_test).argmax(axis=1)
print(metrics.classification_report(y_predict,y_test,digits=5))



history=model.fit(x_train,y_train,epochs=100,validation_data=(x_test,y_test),verbose=0)
y_predict=model.predict(x_test).argmax(axis=1)
print(metrics.classification_report(y_predict,y_test,digits=5))
history=model.fit(x_train,y_train,epochs=100,validation_data=(x_test,y_test),verbose=0)
y_predict=model.predict(x_test).argmax(axis=1)
print(metrics.classification_report(y_predict,y_test,digits=5))



history=model.fit(x_train,y_train,epochs=100,validation_data=(x_test,y_test),verbose=0)
y_predict=model.predict(x_test).argmax(axis=1)
print(metrics.classification_report(y_predict,y_test,digits=5))
history=model.fit(x_train,y_train,epochs=100,validation_data=(x_test,y_test),verbose=0)
y_predict=model.predict(x_test).argmax(axis=1)
print(metrics.classification_report(y_predict,y_test,digits=5))



history=model.fit(x_train,y_train,epochs=100,validation_data=(x_test,y_test),verbose=0)
y_predict=model.predict(x_test).argmax(axis=1)
print(metrics.classification_report(y_predict,y_test,digits=5))
history=model.fit(x_train,y_train,epochs=100,validation_data=(x_test,y_test),verbose=0)
y_predict=model.predict(x_test).argmax(axis=1)
print(metrics.classification_report(y_predict,y_test,digits=5))




              precision    recall  f1-score   support

           0    0.99588   0.99087   0.99337      9746
           1    0.81481   0.95652   0.88000        23
           2    0.86047   0.87264   0.86651       212
           3    0.58209   0.92857   0.71560        42
           4    0.60000   0.84000   0.70000        25
           5    0.80723   0.84277   0.82462       159

    accuracy                        0.98540     10207
   macro avg    0.77675   0.90523   0.83001     10207
weighted avg    0.98704   0.98540   0.98598     10207

              precision    recall  f1-score   support

           0    0.99639   0.99047   0.99342      9755
           1    0.74074   0.95238   0.83333        21
           2    0.84186   0.87864   0.85986       206
           3    0.62687   0.91304   0.74336        46
           4    0.57143   0.95238   0.71429        21
           5    0.82530   0.86709   0.84568       158

    accuracy                        0.98579     10207
   macro avg    0.76710

In [10]:
#Essai avec deux seed différents

In [31]:
tf.random.set_seed(2)
model = build_model2(28, 28, 3, 6,0.3)
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy",metrics=["acc"])

history=model.fit(x_train,y_train,epochs=100,validation_data=(x_test,y_test),verbose=0)
y_predict=model.predict(x_test).argmax(axis=1)
print(metrics.classification_report(y_predict,y_test,digits=5))
history=model.fit(x_train,y_train,epochs=100,validation_data=(x_test,y_test),verbose=0)
y_predict=model.predict(x_test).argmax(axis=1)
print(metrics.classification_report(y_predict,y_test,digits=5))


              precision    recall  f1-score   support

           0    0.99783   0.98987   0.99384      9775
           1    0.74074   1.00000   0.85106        20
           2    0.87442   0.89952   0.88679       209
           3    0.59701   0.93023   0.72727        43
           4    0.54286   0.95000   0.69091        20
           5    0.76506   0.90714   0.83007       140

    accuracy                        0.98658     10207
   macro avg    0.75299   0.94613   0.82999     10207
weighted avg    0.98903   0.98658   0.98740     10207

              precision    recall  f1-score   support

           0    0.99804   0.98896   0.99348      9786
           1    0.74074   1.00000   0.85106        20
           2    0.82326   0.89848   0.85922       197
           3    0.64179   0.95556   0.76786        45
           4    0.51429   1.00000   0.67925        18
           5    0.77711   0.91489   0.84039       141

    accuracy                        0.98609     10207
   macro avg    0.74920

In [32]:
tf.random.set_seed(3)
model = build_model2(28, 28, 3, 6,0.3)
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy",metrics=["acc"])

history=model.fit(x_train,y_train,epochs=100,validation_data=(x_test,y_test),verbose=0)
y_predict=model.predict(x_test).argmax(axis=1)
print(metrics.classification_report(y_predict,y_test,digits=5))
history=model.fit(x_train,y_train,epochs=100,validation_data=(x_test,y_test),verbose=0)
y_predict=model.predict(x_test).argmax(axis=1)
print(metrics.classification_report(y_predict,y_test,digits=5))


              precision    recall  f1-score   support

           0    0.99691   0.99007   0.99347      9764
           1    0.85185   0.95833   0.90196        24
           2    0.80930   0.92063   0.86139       189
           3    0.71642   0.84211   0.77419        57
           4    0.57143   0.83333   0.67797        24
           5    0.77711   0.86577   0.81905       149

    accuracy                        0.98570     10207
   macro avg    0.78717   0.90171   0.83800     10207
weighted avg    0.98732   0.98570   0.98630     10207

              precision    recall  f1-score   support

           0    0.99691   0.99057   0.99373      9759
           1    0.77778   0.95455   0.85714        22
           2    0.84651   0.87923   0.86256       207
           3    0.67164   0.86538   0.75630        52
           4    0.57143   1.00000   0.72727        20
           5    0.80120   0.90476   0.84984       147

    accuracy                        0.98638     10207
   macro avg    0.77758

In [10]:
#Avec l'autre modèle qui est un peu plus profond et avec beaucoup plus de dropout d'ailleurs

In [30]:
model = build_model1(28, 28, 3, 6)
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy",metrics=["acc"])

history=model.fit(x_train,y_train,epochs=100,validation_data=(x_test,y_test),verbose=0)
y_predict=model.predict(x_test).argmax(axis=1)
print(metrics.classification_report(y_predict,y_test,digits=5))
history=model.fit(x_train,y_train,epochs=100,validation_data=(x_test,y_test),verbose=0)
y_predict=model.predict(x_test).argmax(axis=1)
print(metrics.classification_report(y_predict,y_test,digits=5))



history=model.fit(x_train,y_train,epochs=100,validation_data=(x_test,y_test),verbose=0)
y_predict=model.predict(x_test).argmax(axis=1)
print(metrics.classification_report(y_predict,y_test,digits=5))
history=model.fit(x_train,y_train,epochs=100,validation_data=(x_test,y_test),verbose=0)
y_predict=model.predict(x_test).argmax(axis=1)
print(metrics.classification_report(y_predict,y_test,digits=5))



history=model.fit(x_train,y_train,epochs=100,validation_data=(x_test,y_test),verbose=0)
y_predict=model.predict(x_test).argmax(axis=1)
print(metrics.classification_report(y_predict,y_test,digits=5))
history=model.fit(x_train,y_train,epochs=100,validation_data=(x_test,y_test),verbose=0)
y_predict=model.predict(x_test).argmax(axis=1)
print(metrics.classification_report(y_predict,y_test,digits=5))



history=model.fit(x_train,y_train,epochs=100,validation_data=(x_test,y_test),verbose=0)
y_predict=model.predict(x_test).argmax(axis=1)
print(metrics.classification_report(y_predict,y_test,digits=5))
history=model.fit(x_train,y_train,epochs=100,validation_data=(x_test,y_test),verbose=0)
y_predict=model.predict(x_test).argmax(axis=1)
print(metrics.classification_report(y_predict,y_test,digits=5))



history=model.fit(x_train,y_train,epochs=100,validation_data=(x_test,y_test),verbose=0)
y_predict=model.predict(x_test).argmax(axis=1)
print(metrics.classification_report(y_predict,y_test,digits=5))
history=model.fit(x_train,y_train,epochs=100,validation_data=(x_test,y_test),verbose=0)
y_predict=model.predict(x_test).argmax(axis=1)
print(metrics.classification_report(y_predict,y_test,digits=5))




              precision    recall  f1-score   support

           0    0.99608   0.99148   0.99378      9742
           1    0.70370   0.95000   0.80851        20
           2    0.86977   0.86574   0.86775       216
           3    0.62687   0.82353   0.71186        51
           4    0.57143   0.80000   0.66667        25
           5    0.82530   0.89542   0.85893       153

    accuracy                        0.98599     10207
   macro avg    0.76552   0.88770   0.81792     10207
weighted avg    0.98739   0.98599   0.98651     10207

              precision    recall  f1-score   support

           0    0.99557   0.99087   0.99321      9743
           1    0.70370   1.00000   0.82609        19
           2    0.86977   0.91667   0.89260       204
           3    0.61194   0.80392   0.69492        51
           4    0.54286   0.73077   0.62295        26
           5    0.84337   0.85366   0.84848       164

    accuracy                        0.98560     10207
   macro avg    0.76120

In [10]:
#Sans preprocess input

In [27]:
from numpy.random import seed
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
seed(1)
tf.random.set_seed(2)

data_train,data_test= train_test_split(df,stratify=df["class"], test_size=test_size,random_state=42)
train_data_generator = ImageDataGenerator()

test_data_generator = ImageDataGenerator()
    #preprocessing_function = preprocess_input)




train_generator = train_data_generator.flow_from_dataframe(dataframe=data_train,
                                                          directory="",
                                                           x_col = "img_paths",
                                                           class_mode ="sparse",
                                                          target_size = (28 , 28), 
                                                          batch_size = len(data_train) )


test_generator = test_data_generator.flow_from_dataframe(dataframe=data_test,
                                                          directory="",
                                                           x_col = "img_paths",
                                                           class_mode ="sparse",
                                                          target_size = (28 , 28), 
                                                          batch_size = len(data_test))

gen=train_generator[0]

x_train=gen[0]
y_train=gen[1]



gen_test=test_generator[0]

x_test=gen_test[0]
y_test=gen_test[1]


Found 40827 validated image filenames belonging to 6 classes.
Found 10207 validated image filenames belonging to 6 classes.


In [28]:
model = build_model2(28, 28, 3, 6,0.3)
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy",metrics=["acc"])

history=model.fit(x_train,y_train,epochs=100,validation_data=(x_test,y_test),verbose=0)
y_predict=model.predict(x_test).argmax(axis=1)
print(metrics.classification_report(y_predict,y_test,digits=5))
history=model.fit(x_train,y_train,epochs=100,validation_data=(x_test,y_test),verbose=0)
y_predict=model.predict(x_test).argmax(axis=1)
print(metrics.classification_report(y_predict,y_test,digits=5))



history=model.fit(x_train,y_train,epochs=100,validation_data=(x_test,y_test),verbose=0)
y_predict=model.predict(x_test).argmax(axis=1)
print(metrics.classification_report(y_predict,y_test,digits=5))
history=model.fit(x_train,y_train,epochs=100,validation_data=(x_test,y_test),verbose=0)
y_predict=model.predict(x_test).argmax(axis=1)
print(metrics.classification_report(y_predict,y_test,digits=5))



history=model.fit(x_train,y_train,epochs=100,validation_data=(x_test,y_test),verbose=0)
y_predict=model.predict(x_test).argmax(axis=1)
print(metrics.classification_report(y_predict,y_test,digits=5))
history=model.fit(x_train,y_train,epochs=100,validation_data=(x_test,y_test),verbose=0)
y_predict=model.predict(x_test).argmax(axis=1)
print(metrics.classification_report(y_predict,y_test,digits=5))



history=model.fit(x_train,y_train,epochs=100,validation_data=(x_test,y_test),verbose=0)
y_predict=model.predict(x_test).argmax(axis=1)
print(metrics.classification_report(y_predict,y_test,digits=5))
history=model.fit(x_train,y_train,epochs=100,validation_data=(x_test,y_test),verbose=0)
y_predict=model.predict(x_test).argmax(axis=1)
print(metrics.classification_report(y_predict,y_test,digits=5))



history=model.fit(x_train,y_train,epochs=100,validation_data=(x_test,y_test),verbose=0)
y_predict=model.predict(x_test).argmax(axis=1)
print(metrics.classification_report(y_predict,y_test,digits=5))
history=model.fit(x_train,y_train,epochs=100,validation_data=(x_test,y_test),verbose=0)
y_predict=model.predict(x_test).argmax(axis=1)
print(metrics.classification_report(y_predict,y_test,digits=5))




              precision    recall  f1-score   support

           0    0.99588   0.99158   0.99372      9739
           1    0.66667   0.90000   0.76596        20
           2    0.87907   0.87500   0.87703       216
           3    0.67164   0.86538   0.75630        52
           4    0.60000   0.91304   0.72414        23
           5    0.82530   0.87261   0.84830       157

    accuracy                        0.98628     10207
   macro avg    0.77309   0.90294   0.82757     10207
weighted avg    0.98759   0.98628   0.98675     10207

              precision    recall  f1-score   support

           0    0.99722   0.98855   0.99286      9782
           1    0.59259   1.00000   0.74419        16
           2    0.86047   0.88517   0.87264       209
           3    0.59701   0.86957   0.70796        46
           4    0.60000   1.00000   0.75000        21
           5    0.73494   0.91729   0.81605       133

    accuracy                        0.98501     10207
   macro avg    0.73037

In [33]:
tf.random.set_seed(2)
model = build_model(28, 28, 3, 6,0.3)
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy",metrics=["acc"])
history=model.fit(x_train,y_train,epochs=10,validation_data=(x_test,y_test),verbose=0)
y_predict=model.predict(x_test).argmax(axis=1)
print(metrics.classification_report(y_predict,y_test,digits=5))

              precision    recall  f1-score   support

           0    0.99423   0.99116   0.99269      9727
           1    0.62963   0.94444   0.75556        18
           2    0.88372   0.82251   0.85202       231
           3    0.62687   0.91304   0.74336        46
           4    0.60000   0.84000   0.70000        25
           5    0.80723   0.83750   0.82209       160

    accuracy                        0.98413     10207
   macro avg    0.75695   0.89144   0.81095     10207
weighted avg    0.98553   0.98413   0.98457     10207



In [34]:
tf.random.set_seed(2)
model = build_model(28, 28, 3, 6,0.3)
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy",metrics=["acc"])
history=model.fit(x_train,y_train,epochs=5,validation_data=(x_test,y_test),verbose=0)
y_predict=model.predict(x_test).argmax(axis=1)
print(metrics.classification_report(y_predict,y_test,digits=5))


history=model.fit(x_train,y_train,epochs=5,validation_data=(x_test,y_test),verbose=0)
y_predict=model.predict(x_test).argmax(axis=1)

              precision    recall  f1-score   support

           0    0.99247   0.98799   0.99023      9741
           1    0.40741   1.00000   0.57895        11
           2    0.76279   0.85864   0.80788       191
           3    0.65672   0.69841   0.67692        63
           4    0.42857   0.93750   0.58824        16
           5    0.78313   0.70270   0.74074       185

    accuracy                        0.97854     10207
   macro avg    0.67185   0.86421   0.73049     10207
weighted avg    0.98079   0.97854   0.97928     10207



In [35]:
print(metrics.classification_report(y_predict,y_test,digits=5))

              precision    recall  f1-score   support

           0    0.99639   0.98572   0.99103      9802
           1    0.62963   0.94444   0.75556        18
           2    0.75349   0.91525   0.82653       177
           3    0.76119   0.78462   0.77273        65
           4    0.45714   0.94118   0.61538        17
           5    0.68072   0.88281   0.76871       128

    accuracy                        0.98178     10207
   macro avg    0.71309   0.90900   0.78832     10207
weighted avg    0.98518   0.98178   0.98295     10207



In [38]:
tf.random.set_seed(2)
model = build_model(28, 28, 3, 6,0.3)
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy",metrics=["acc"])
history=model.fit(x_train,y_train,epochs=1,validation_data=(x_test,y_test),verbose=0)
y_predict=model.predict(x_test).argmax(axis=1)
print(metrics.classification_report(y_predict,y_test,digits=5))


history=model.fit(x_train,y_train,epochs=1,validation_data=(x_test,y_test),verbose=0)
y_predict=model.predict(x_test).argmax(axis=1)
print(metrics.classification_report(y_predict,y_test,digits=5))


/home/marcpozzo/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


              precision    recall  f1-score   support

         0.0    0.98969   0.96851   0.97899      9909
         1.0    0.07407   0.66667   0.13333         3
         2.0    0.62791   0.61644   0.62212       219
         3.0    0.05970   0.80000   0.11111         5
         4.0    0.00000   0.00000   0.00000         0
         5.0    0.19277   0.45070   0.27004        71

    accuracy                        0.95719     10207
   macro avg    0.32402   0.58372   0.35260     10207
weighted avg    0.97566   0.95719   0.96572     10207

              precision    recall  f1-score   support

         0.0    0.99701   0.97273   0.98472      9939
         1.0    0.14815   1.00000   0.25806         4
         2.0    0.60000   0.81646   0.69169       158
         3.0    0.29851   0.86957   0.44444        23
         4.0    0.00000   0.00000   0.00000         0
         5.0    0.44578   0.89157   0.59438        83

    accuracy                        0.96943     10207
   macro avg    0.41491

In [39]:
tf.random.set_seed(2)
model = build_model(28, 28, 3, 6,0.3)
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy",metrics=["acc"])
history=model.fit(x_train,y_train,epochs=1,validation_data=(x_test,y_test),verbose=0)
y_predict=model.predict(x_test).argmax(axis=1)
print(metrics.classification_report(y_predict,y_test,digits=5))


history=model.fit(x_train,y_train,epochs=1,validation_data=(x_test,y_test),verbose=0)
y_predict=model.predict(x_test).argmax(axis=1)
print(metrics.classification_report(y_predict,y_test,digits=5))

/home/marcpozzo/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


              precision    recall  f1-score   support

         0.0    0.98969   0.96851   0.97899      9909
         1.0    0.07407   0.66667   0.13333         3
         2.0    0.62791   0.61644   0.62212       219
         3.0    0.05970   0.80000   0.11111         5
         4.0    0.00000   0.00000   0.00000         0
         5.0    0.19277   0.45070   0.27004        71

    accuracy                        0.95719     10207
   macro avg    0.32402   0.58372   0.35260     10207
weighted avg    0.97566   0.95719   0.96572     10207

              precision    recall  f1-score   support

         0.0    0.99701   0.97273   0.98472      9939
         1.0    0.14815   1.00000   0.25806         4
         2.0    0.60000   0.81646   0.69169       158
         3.0    0.29851   0.86957   0.44444        23
         4.0    0.00000   0.00000   0.00000         0
         5.0    0.44578   0.89157   0.59438        83

    accuracy                        0.96943     10207
   macro avg    0.41491

In [43]:
val_acc=[]
acc=[]
val_acc.append(history.history["val_acc"])
acc.append(history.history["acc"])


tf.random.set_seed(2)
model = build_model(28, 28, 3, 6,0.3)
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy",metrics=["acc"])
history=model.fit(x_train,y_train,epochs=1,validation_data=(x_test,y_test),verbose=0)
y_predict=model.predict(x_test).argmax(axis=1)
print(metrics.classification_report(y_predict,y_test,digits=5))

tf.random.set_seed(2)
history=model.fit(x_train,y_train,epochs=1,validation_data=(x_test,y_test),verbose=0)
y_predict=model.predict(x_test).argmax(axis=1)
print(metrics.classification_report(y_predict,y_test,digits=5))

/home/marcpozzo/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


              precision    recall  f1-score   support

         0.0    0.98969   0.96851   0.97899      9909
         1.0    0.07407   0.66667   0.13333         3
         2.0    0.62791   0.61644   0.62212       219
         3.0    0.05970   0.80000   0.11111         5
         4.0    0.00000   0.00000   0.00000         0
         5.0    0.19277   0.45070   0.27004        71

    accuracy                        0.95719     10207
   macro avg    0.32402   0.58372   0.35260     10207
weighted avg    0.97566   0.95719   0.96572     10207

              precision    recall  f1-score   support

         0.0    0.99567   0.97270   0.98405      9926
         1.0    0.33333   0.90000   0.48649        10
         2.0    0.62326   0.74860   0.68020       179
         3.0    0.38806   0.89655   0.54167        29
         4.0    0.00000   0.00000   0.00000         0
         5.0    0.33133   0.87302   0.48035        63

    accuracy                        0.96787     10207
   macro avg    0.44527

In [44]:
val_acc=[]
acc=[]

tf.random.set_seed(2)
model = build_model(28, 28, 3, 6,0.3)
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy",metrics=["acc"])
history=model.fit(x_train,y_train,epochs=2,validation_data=(x_test,y_test),verbose=0)
val_acc.append(history.history["val_acc"])
acc.append(history.history["acc"])
y_predict=model.predict(x_test).argmax(axis=1)
print(metrics.classification_report(y_predict,y_test,digits=5))



              precision    recall  f1-score   support

         0.0    0.99392   0.97848   0.98614      9850
         1.0    0.00000   0.00000   0.00000         0
         2.0    0.63256   0.78161   0.69923       174
         3.0    0.40299   0.79412   0.53465        34
         4.0    0.02857   1.00000   0.05556         1
         5.0    0.66265   0.74324   0.70064       148

    accuracy                        0.97110     10207
   macro avg    0.45345   0.71624   0.49604     10207
weighted avg    0.98089   0.97110   0.97551     10207



/home/marcpozzo/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [53]:
val_acc=[]
acc=[]

tf.random.set_seed(2)
model = build_model(28, 28, 3, 6,0.3)
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy",metrics=["acc"])
history=model.fit(x_train,y_train,epochs=2,validation_data=(x_test,y_test))
val_acc.append(history.history["val_acc"])
acc.append(history.history["acc"])
y_predict=model.predict(x_test).argmax(axis=1)
print(metrics.classification_report(y_predict,y_test,digits=5))

history=model.fit(x_train,y_train,epochs=2,validation_data=(x_test,y_test))
val_acc.append(history.history["val_acc"])
acc.append(history.history["acc"])
y_predict=model.predict(x_test).argmax(axis=1)
print(metrics.classification_report(y_predict,y_test,digits=5))

Train on 40827 samples, validate on 10207 samples
Epoch 1/2
40827/40827 [==============================] - 24s 585us/sample - loss: 0.3504 - acc: 0.9061 - val_loss: 0.1371 - val_acc: 0.9572
Epoch 2/2
40827/40827 [==============================] - 23s 570us/sample - loss: 0.1438 - acc: 0.9614 - val_loss: 0.1041 - val_acc: 0.9711


/home/marcpozzo/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


              precision    recall  f1-score   support

         0.0    0.99392   0.97848   0.98614      9850
         1.0    0.00000   0.00000   0.00000         0
         2.0    0.63256   0.78161   0.69923       174
         3.0    0.40299   0.79412   0.53465        34
         4.0    0.02857   1.00000   0.05556         1
         5.0    0.66265   0.74324   0.70064       148

    accuracy                        0.97110     10207
   macro avg    0.45345   0.71624   0.49604     10207
weighted avg    0.98089   0.97110   0.97551     10207

Train on 40827 samples, validate on 10207 samples
Epoch 1/2
40827/40827 [==============================] - 23s 571us/sample - loss: 0.1082 - acc: 0.9686 - val_loss: 0.1030 - val_acc: 0.9693
Epoch 2/2
40827/40827 [==============================] - 23s 570us/sample - loss: 0.0920 - acc: 0.9728 - val_loss: 0.0727 - val_acc: 0.9778
              precision    recall  f1-score   support

           0    0.99742   0.98143   0.98936      9855
           1    0.

In [54]:
val_acc=[]
acc=[]

tf.random.set_seed(2)
model = build_model(28, 28, 3, 6,0.3)
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy",metrics=["acc"])
history=model.fit(x_train,y_train,epochs=2,validation_data=(x_test,y_test))
val_acc.append(history.history["val_acc"])
acc.append(history.history["acc"])
y_predict=model.predict(x_test).argmax(axis=1)
print(metrics.classification_report(y_predict,y_test,digits=5))

tf.random.set_seed(2)
history=model.fit(x_train,y_train,epochs=2,validation_data=(x_test,y_test))
val_acc.append(history.history["val_acc"])
acc.append(history.history["acc"])
y_predict=model.predict(x_test).argmax(axis=1)
print(metrics.classification_report(y_predict,y_test,digits=5))

Train on 40827 samples, validate on 10207 samples
Epoch 1/2
40827/40827 [==============================] - 24s 585us/sample - loss: 0.3504 - acc: 0.9061 - val_loss: 0.1371 - val_acc: 0.9572
Epoch 2/2
40827/40827 [==============================] - 23s 575us/sample - loss: 0.1438 - acc: 0.9614 - val_loss: 0.1041 - val_acc: 0.9711


/home/marcpozzo/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


              precision    recall  f1-score   support

         0.0    0.99392   0.97848   0.98614      9850
         1.0    0.00000   0.00000   0.00000         0
         2.0    0.63256   0.78161   0.69923       174
         3.0    0.40299   0.79412   0.53465        34
         4.0    0.02857   1.00000   0.05556         1
         5.0    0.66265   0.74324   0.70064       148

    accuracy                        0.97110     10207
   macro avg    0.45345   0.71624   0.49604     10207
weighted avg    0.98089   0.97110   0.97551     10207

Train on 40827 samples, validate on 10207 samples
Epoch 1/2
40827/40827 [==============================] - 23s 572us/sample - loss: 0.1037 - acc: 0.9698 - val_loss: 0.1075 - val_acc: 0.9689
Epoch 2/2
40827/40827 [==============================] - 23s 572us/sample - loss: 0.0861 - acc: 0.9739 - val_loss: 0.0738 - val_acc: 0.9788
              precision    recall  f1-score   support

           0    0.99732   0.98263   0.98992      9842
           1    0.

In [50]:
val_acc=[]
acc=[]

tf.random.set_seed(2)
model = build_model(28, 28, 3, 6,0.3)
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy",metrics=["acc"])
history=model.fit(x_train,y_train,epochs=4,validation_data=(x_test,y_test))
val_acc.append(history.history["val_acc"])
acc.append(history.history["acc"])
y_predict=model.predict(x_test).argmax(axis=1)
print(metrics.classification_report(y_predict,y_test,digits=5))


Train on 40827 samples, validate on 10207 samples
Epoch 1/4
40827/40827 [==============================] - 24s 584us/sample - loss: 0.3504 - acc: 0.9061 - val_loss: 0.1371 - val_acc: 0.9572
Epoch 2/4
40827/40827 [==============================] - 23s 569us/sample - loss: 0.1438 - acc: 0.9614 - val_loss: 0.1041 - val_acc: 0.9711
Epoch 3/4
40827/40827 [==============================] - 23s 574us/sample - loss: 0.1116 - acc: 0.9672 - val_loss: 0.1053 - val_acc: 0.9675
Epoch 4/4
40827/40827 [==============================] - 23s 572us/sample - loss: 0.0934 - acc: 0.9715 - val_loss: 0.0741 - val_acc: 0.9783
              precision    recall  f1-score   support

           0    0.99639   0.98241   0.98935      9835
           1    0.40741   0.84615   0.55000        13
           2    0.79070   0.88542   0.83538       192
           3    0.40299   0.84375   0.54545        32
           4    0.40000   0.93333   0.56000        15
           5    0.61446   0.85000   0.71329       120

    accura

In [10]:
#Ca c'est plus ou moins les anciennes versions 

In [10]:
#Nouveau
tf.random.set_seed(2)
model = build_model(28, 28, 3, 6,0.3)
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy",metrics=["acc"])
history=model.fit(x_train,y_train,epochs=200,validation_data=(x_test,y_test),verbose=0)
y_predict=model.predict(x_test).argmax(axis=1)
print(metrics.classification_report(y_predict,y_test,digits=5))
model.save("test_200ep")
#model.save("tf_fp_100ep")

              precision    recall  f1-score   support

           0    0.99783   0.98886   0.99333      9785
           1    0.66667   0.94737   0.78261        19
           2    0.81860   0.88889   0.85230       198
           3    0.59701   0.93023   0.72727        43
           4    0.54286   0.95000   0.69091        20
           5    0.79518   0.92958   0.85714       142

    accuracy                        0.98570     10207
   macro avg    0.73636   0.93915   0.81726     10207
weighted avg    0.98834   0.98570   0.98659     10207



In [19]:
#Nouveau
tf.random.set_seed(2)

history=model.fit(x_train,y_train,epochs=100,validation_data=(x_test,y_test),verbose=0)
y_predict=model.predict(x_test).argmax(axis=1)
print(metrics.classification_report(y_predict,y_test,digits=5))
model.save("test_300ep")
#model.save("tf_fp_100ep")

              precision    recall  f1-score   support

           0    0.99629   0.99506   0.99567      9709
           1    0.85185   0.95833   0.90196        24
           2    0.89302   0.91429   0.90353       210
           3    0.83582   0.93333   0.88189        60
           4    0.85714   0.96774   0.90909        31
           5    0.90361   0.86705   0.88496       173

    accuracy                        0.99069     10207
   macro avg    0.88962   0.93930   0.91285     10207
weighted avg    0.99089   0.99069   0.99075     10207

INFO:tensorflow:Assets written to: test_300ep/assets


In [ ]:
        precision    recall  f1-score   support

           0    0.99783   0.98987   0.99384      9775
           1    0.74074   1.00000   0.85106        20
           2    0.87442   0.89952   0.88679       209
           3    0.59701   0.93023   0.72727        43
           4    0.54286   0.95000   0.69091        20
           5    0.76506   0.90714   0.83007       140

    accuracy                        0.98658     10207
   macro avg    0.75299   0.94613   0.82999     10207
weighted avg    0.98903   0.98658   0.98740     10207

In [ ]:
   precision    recall  f1-score   support

           0    0.99649   0.99250   0.99449      9736
           1    0.88889   0.85714   0.87273        28
           2    0.90233   0.87783   0.88991       221
           3    0.68657   0.92000   0.78632        50
           4    0.68571   0.88889   0.77419        27
           5    0.79518   0.91034   0.84887       145

    accuracy                        0.98785     10207
   macro avg    0.82586   0.90778   0.86109     10207
weighted avg    0.98896   0.98785   0.98822     10207

              precision    recall  f1-score   support

           0    0.99031   0.99803   0.99415      9622
           1    0.88889   0.88889   0.88889        27
           2    0.96744   0.80309   0.87764       259
           3    0.97015   0.78313   0.86667        83
           4    0.91429   0.86486   0.88889        37
           5    0.92771   0.86034   0.89275       179

    accuracy                        0.98815     10207
   macro avg    0.94313   0.86639   0.90150     10207
weighted avg    0.98792   0.98815   0.98772     10207

